In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import os


In [ ]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

train_dataset  = dsets.MNIST(root='./data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                          train=False,
                          transform=transforms.ToTensor())

train_loader  = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2), #1 * 28 * 28 -> 16 * 28 * 28
            nn.BatchNorm2d(16), #16 * 28 * 28
            nn.ReLU(),                 #16 * 28 * 28
            nn.MaxPool2d(2))       # 16 * 14 * 14
        self.layer2  = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2), # 16 * 14 * 14 -> 32 * 14 * 14
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()) # 32 * 7 * 7
        self.fc = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.fc(out)
        return out
    
cnn = CNN()

In [ ]:

criterion = nn.CrossEntropyLoss() # Loss
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                  %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data))
        


Epoch [1/5], lter [100/600] Loss: 0.1255
Epoch [1/5], lter [200/600] Loss: 0.1426
Epoch [1/5], lter [300/600] Loss: 0.1203
Epoch [1/5], lter [400/600] Loss: 0.0539
Epoch [1/5], lter [500/600] Loss: 0.0825
Epoch [1/5], lter [600/600] Loss: 0.0310
Epoch [2/5], lter [100/600] Loss: 0.0662
Epoch [2/5], lter [200/600] Loss: 0.0551
Epoch [2/5], lter [300/600] Loss: 0.0925
Epoch [2/5], lter [400/600] Loss: 0.0048
Epoch [2/5], lter [500/600] Loss: 0.0183
Epoch [2/5], lter [600/600] Loss: 0.1509
Epoch [3/5], lter [100/600] Loss: 0.0360
Epoch [3/5], lter [200/600] Loss: 0.0359
Epoch [3/5], lter [300/600] Loss: 0.0200
Epoch [3/5], lter [400/600] Loss: 0.0399
Epoch [3/5], lter [500/600] Loss: 0.0338
Epoch [3/5], lter [600/600] Loss: 0.0026
Epoch [4/5], lter [100/600] Loss: 0.0199
Epoch [4/5], lter [200/600] Loss: 0.0368
Epoch [4/5], lter [300/600] Loss: 0.0073
Epoch [4/5], lter [400/600] Loss: 0.0251
Epoch [4/5], lter [500/600] Loss: 0.0234
Epoch [4/5], lter [600/600] Loss: 0.0093
Epoch [5/5], lte

In [ ]:
cnn.eval()
correct = 0
total = 0
for images, labels in test_loader:

    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('test Accuracy 10000 test images  = %f %%' % (100 * correct/total))

test Accuracy 10000 test images  = 98.779999 %
